**CNN modelis.** Naudojama duomenų augmentacija, Sobel filtras, pre-trained modelis.

# Naudojamos bibliotekos #

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split, ConcatDataset

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from scipy.ndimage import sobel
import pickle, gzip

# Sutvarkome failus ir duomenis

In [2]:
from google.colab import files
uploaded = files.upload() # Failų įkėlimas. flatland_train, flatland_test ir model_pth (pretrained modelis)

Saving flatland_test.data to flatland_test (2).data
Saving flatland_train.data to flatland_train (2).data
Saving model.pth to model (2).pth


In [58]:
X, y = pickle.load(gzip.open("flatland_train.data", 'rb'))
X_eval, y_eval = pickle.load(gzip.open("flatland_test.data", 'rb'))
y[y!=0] -= 2 # laikysime, kad apskritimas=0, trikampis=1, kvadratas=2 ir t.t.
X = torch.tensor(X, dtype=torch.float32) / 255 # normalizuojame
X_eval = torch.tensor(X_eval, dtype=torch.float32) / 255 # normalizuojame
y = torch.tensor(y, dtype=torch.long)

# Naudosime GPU dėl greičio#

In [59]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#Sobel filtras, kad išryškėtų briaunos ir kampai#

In [60]:
def apply_sobel(X):
    X_sobel = np.zeros_like(X)
    for i in range(X.shape[0]):
        img = X[i].numpy()
        sobel_x = sobel(img, axis=1)
        sobel_y = sobel(img, axis=0)
        sobel_img = np.hypot(sobel_x, sobel_y)
        X_sobel[i] = torch.tensor(sobel_img)
    return torch.tensor(X_sobel, dtype=torch.float32)

X = apply_sobel(X)
#Pritaikome filtrą ir eval duomenims
X_eval = apply_sobel(X_eval)

# Augmentacija (pridedame apverstus image'us)

In [71]:
X = torch.cat((X, torch.flip(X, dims=[1]), torch.flip(torch.flip(X, dims=[1]),
        dims=[2])), dim=0) #augmentuojame diagonal ir horizontal flip'us

y = torch.cat((y, y, y), dim=0) #papildome label'ius

#train ir test split'as

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

#prijungiame grayscale chanel
X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

train_dataset_ = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset_, batch_size=128, shuffle=True) #batchsize=128
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# CNN architektūra. Naudojama 3 fc, max pooling, drop'outas (be drop'outo ir su 2-3 epochomis modelis praktiškai turi tą patį tikslumą ~0.5% skirtumas)

In [63]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(128*6*6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 5)

        self.dropout = nn.Dropout(p=0.4)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128*6*6)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Inicijuojame modelį su criterion=CrossEntropy, optimizer=SGD. Užload'inam pretrained modelį.

In [64]:
model = CNNModel().to(device)
model.load_state_dict(torch.load("model.pth", map_location=torch.device('cpu')))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.92)

<ipython-input-64-f48f081b88a0>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth", map_location=torch.device('cpu')))


# Treniravimas ir testavimas #

In [65]:
epochs = 10 #TRENIRUOTI TIK SU GPU, NES CPU 1 EPOCHA UŽTRUNKA 5MIN

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    epoch_train_loss = running_loss / len(train_loader)
    epoch_train_acc = correct_train / total_train * 100

    #Modelio įvertinimas su test data
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad(): #nereikia skaičiuoti gradientų
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted_val = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()

    epoch_val_acc = 100 * correct_val / total_val

    print(f"Epoch: {epoch + 1}; Train Loss: {epoch_train_loss:.3f}")
    print(f"Train Acc: {epoch_train_acc:.2f}%")
print(f"Final Validation Acc: {epoch_val_acc:.2f}%")

Epoch: 1; Train Loss: 0.757
Train Acc: 78.67%
Epoch: 2; Train Loss: 0.182
Train Acc: 96.60%
Epoch: 3; Train Loss: 0.159
Train Acc: 97.51%
Epoch: 4; Train Loss: 0.147
Train Acc: 97.77%
Epoch: 5; Train Loss: 0.134
Train Acc: 98.04%
Epoch: 6; Train Loss: 0.129
Train Acc: 98.09%
Epoch: 7; Train Loss: 0.124
Train Acc: 98.09%
Epoch: 8; Train Loss: 0.122
Train Acc: 98.17%
Epoch: 9; Train Loss: 0.115
Train Acc: 98.22%
Epoch: 10; Train Loss: 0.113
Train Acc: 98.19%
Final Validation Acc: 98.53%


# Modelio spėjimas ant X_eval duomenų#

In [66]:
eval_loader = torch.utils.data.DataLoader(X_eval.unsqueeze(1), batch_size=128, shuffle=False)

In [67]:
model.eval()
predictions = []

with torch.no_grad():
    for images in eval_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

predictions = np.array(predictions)
predictions[predictions != 0] += 2

In [68]:
#išsaugome spėjimą
with open('predictions.txt', 'w') as f:
    for nr in predictions:
        f.write(str(nr))
files.download('predictions.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [70]:
#išsaugome modelį
torch.save(model.state_dict(), "CNNmodel.pth")
files.download('CNNmodel.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>